In [4]:
# import libs and set options
import pandas as pd 
import numpy as np
import math
import re
import warnings

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('mode.chained_assignment', 'raise')

In [8]:
# define location path
path = "/Users/hoangpham/Downloads/GoodsDistribution/"
path_ho = "/Users/hoangpham/Downloads/GoodsDistribution/HO_Stock.xlsx"

In [9]:
# define dataframe & transform Area, avoid mistakes
df = pd.read_excel(f'{path}Stocks.xlsx')
list_area = df['Area'].values.tolist()

new_area = [re.sub(r'\d+', "", re.sub(r'\s+', "", area)) for area in list_area]
df = df.drop(['Area'], axis =1)
df['Area'] = new_area

list_area_2 = df['Area'].drop_duplicates().values.tolist()

['ĐàNẵng', 'HàNội', 'HồChíMinh', 'MiềnĐông', 'MiềnTây']

In [53]:
list(df.iterrows())[0][1]['Area']

'ĐàNẵng'

In [4]:
df_1 = pd.read_excel(path_ho)

df_2 = [df[df.Area == list_area_2[area]] for area in range(0,len(list_area_2))]

In [5]:
# define min & max DOS (optimize 30 days)
min_day = 26
max_day = 34

# define drop columns, sort orders, classification good's statement function
def drop_sub_columns_pool1(df):
    df = df.drop(['StockQuantity', 'SO1', 'SO2', 'SO3', 'SO4', 'AvgSO', 'SellPower', 'DOS', 'Statement'],axis=1)
    return df

def drop_sub_columns_pool2(df):
    df = df.drop(['SellPower', 'DOS', 'Statement', 'Balance_num'],axis=1)
    return df

def sort_by_balance(df):    
    df = df.sort_values(by='Balance_num' ,ascending=False)
    return df

def DOS_Classify (condition):
    if condition < min_day:
        return "Thiếu hàng"
    if condition > max_day:
        return "Thừa hàng"
    if min_day <= condition <= max_day:
        return "Đủ hàng"  

In [6]:
# define distribution function
def allotment(df_1,df_2):

    # if hasattr(df_stock_HO_raw,
    #             'attr_name') and hasattr(df_2,
    #             'attr_name'):

    list_pool_1 = list()
    list_pool_2 = list()
    list_sheet_name_pool_1 =list()
    list_sheet_name_pool_2 =list()
    pool1_tong_hop = pd.DataFrame()
    pool2_tong_hop = pd.DataFrame()
    
    #transform data from df_2 list
    for df in df_2:

        data = df.loc[:,['Area','storeId','storeName', 'productId','productName'
                    ,'SO1','SO2','SO3','SO4', 'AvgSO','StockQuantity']].copy()
        
        #fillna with 0
        data = data.fillna(np.nan).replace([np.nan], 0)
        
        #limit SalePower < 0
        data['AvgSO'] = data['AvgSO'].clip(lower=0)

        #Sell power
        data['SellPower'] = round(data['AvgSO'].div(7),3)
        #DOS: return the quantity fit with sell power
        data['DOS'] = round(data['StockQuantity'] / data['SellPower'],0)

        #DROP residual column df.drop([index])
        #Sort values by DOS
        data = data.sort_values(by=['DOS'], ascending = False)
        #Classify statement
        data['Statement'] = data['DOS'].apply(lambda x: DOS_Classify(x))
        data['Balance_num'] = ''
        
        ### 1st loop
        for i, row in data.iterrows():
            val = row['Statement']
            
            if val == "Thừa hàng":
                data.at[i,'Balance_num']= math.ceil( data['StockQuantity'][i] - data['SellPower'][i] *max_day)   
                
            if val == "Thiếu hàng":
                data.at[i,'Balance_num'] = round(-data['StockQuantity'][i] +data['SellPower'][i] *min_day,0)
                
            if val == "Đủ hàng":
                data.at[i,'Balance_num'] = 0

        # Initialize pool_1 - Dư hàng
        pool_1 = data.loc[data['Statement'] == 'Thừa hàng']
        pool_1 = sort_by_balance(pool_1)
        

        ## Initialize pool_2 - Thiếu hàng
        pool_2 = data.loc[data['Statement'] == 'Thiếu hàng'] 
        pool_2 = sort_by_balance(pool_2)

        # Stores in HCM & HN
        if list(data.iterrows())[0][1]['Area'] == 'HồChíMinh' or list(data.iterrows())[0][1]['Area'] == 'HàNội':
        
            #Setup new pool_1_transfer_clone ; pool_2_get_clone
            pool_1_transfer_clone = pool_1
            pool_2_get_clone = pool_2
        
            # Identify
            # First Initialize cache_transaction
            cache_transaction = pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
            
            # print(cache_transaction)
            # Transaction dataframe
            ## Rule: Compare the productId and Balance number
            
            #define dataframe storage logs transaction and goods to HO (excess stock)
            transaction = pd.DataFrame(columns = ['storeId_transfer','storeName_transfer', 'storeId_receive','storeName_receive','productId','productName','Quantity'])
            
            pool_3_move_to_HO = pd.DataFrame(columns = ['storeId_HO','storeName_HO', 'productId','productName','Quantity'])
            
            while True:
                # The amount of goods exceeded pool_1
                n   =   pool_1_transfer_clone.iloc[0]['Balance_num']
                
                # resolve cache_transaction is blank
                if len(cache_transaction.index) == 0  :
                    d = {'storeId_HO': pool_1_transfer_clone.iloc[0]['storeId'],
                        'storeName_HO': pool_1_transfer_clone.iloc[0]['storeName'], 
                        'productId': pool_1_transfer_clone.iloc[0]['productId'],
                        'productName': pool_1_transfer_clone.iloc[0]['productName'],
                        'Quantity': n}
                    
                    #concat data to pool goods to HO
                    pool_3_move_to_HO = pd.concat([pool_3_move_to_HO, pd.DataFrame(data=d,index=[0])], ignore_index=False)
                    
                    #drop good in first index in transfer pool when cache_transaction is blank
                    pool_1_transfer_clone.drop(index = pool_1_transfer_clone.iloc[0:1,:].index, inplace=True)
                    
                    #break loop when pool_1_transfer is over
                    if len(pool_1_transfer_clone.index) == 0: break

                    #update cache_transaction with new first index items of pool_1_transfer_clone
                    cache_transaction = pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
                    
                    continue
                    
                # The amount of goods lacked pool_2 
                m   =   cache_transaction.iloc[0]['Balance_num']

                ## Case : n >= m; exceed >= lack
                if  n    >=   m :

                    transfering =    n  -   m
                    
                # This dataframe is storing the transaction
                    # create log's dataframe
                    c = {'storeId_transfer': pool_1_transfer_clone.iloc[0]['storeId'],
                            'storeName_transfer': pool_1_transfer_clone.iloc[0]['storeName'], 
                            'storeId_receive':    cache_transaction.iloc[0]['storeId'],
                            'storeName_receive':  cache_transaction.iloc[0]['storeName'],
                            'productId':          pool_1_transfer_clone.iloc[0]['productId'],
                            'productName':        pool_1_transfer_clone.iloc[0]['productName'],'Quantity': m}
                    
                    # concat transaction with log's dataframe
                    transaction =    pd.concat([transaction,pd.DataFrame(data = c, index = [0])], ignore_index=False)

                    # Update values in 2 pools
                    # Update pool_1 - subtract (n - m = transfering)
                    pool_1_transfer_clone.loc[pool_1_transfer_clone.index[0], ['Balance_num']] = transfering
                    
                    # Update pool_2 drop index in pool2
                    pool_2_get_clone.drop(index = cache_transaction.iloc[0:1,:].index, inplace=True)

                    #   ReArrange Pool to identify first index
                    pool_1_transfer_clone = sort_by_balance(pool_1_transfer_clone)
                    pool_2_get_clone = sort_by_balance(pool_2_get_clone)

                    #   Update cache_transaction 
                    # When pool_2_get_clone is blank - don't need to receive goods then break loop
                    if len(pool_2_get_clone.index) == 0:
                        break
                        
                    # update cache
                    cache_transaction = pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
                    continue
                    
                # Case 2 lack > exceed (m > n, n != 0) 
                if m > n and (n != 0 or n != None)  :
    
                    transfering =    m  - n 

                    # This dataframe is storing the transaction

                    e = {'storeId_transfer': pool_1_transfer_clone.iloc[0]['storeId'],
                            'storeName_transfer': pool_1_transfer_clone.iloc[0]['storeName'], 
                            'storeId_receive':    cache_transaction.iloc[0]['storeId'],
                            'storeName_receive':  cache_transaction.iloc[0]['storeName'],
                            'productId':          pool_1_transfer_clone.iloc[0]['productId'],
                            'productName':        pool_1_transfer_clone.iloc[0]['productName'], 'Quantity': n}
                    
                    # Add logs transaction
                    transaction =    pd.concat([transaction,pd.DataFrame(data = e, index=[0])], ignore_index=False)
                    
                    #   Update transfer number and delete row in pool_1
                    pool_2_get_clone.loc[cache_transaction.index[0], ['Balance_num']] = transfering
                    
                    # Update stockQuantity in  pool_2_get_clone
                    pool_2_get_clone.loc[cache_transaction.index[0], ['StockQuantity']] = transfering
                    
                    # drop numbers goods transfer in pool_1
                    pool_1_transfer_clone.drop(index=pool_1_transfer_clone.iloc[0:1,:].index ,inplace=True)
                    
                    #   Refresh pool
                    pool_1_transfer_clone = sort_by_balance(pool_1_transfer_clone)
                    pool_2_get_clone = sort_by_balance(pool_2_get_clone)

                    #   Breake when pool_1 is blank
                    if len(pool_1_transfer_clone.index) == 0 :
                        break
                        
                    #   Update cache transaction    
                    cache_transaction=pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
                    continue
                    
                # Case 3 lack > exceed (m > n) (1 - 0 = 1, case when n = 0 or n = None)
                if m > n and (n == 0 or n == None)  :
    
                    transfering = 0

                    # This dataframe is storing the transaction

                    e = {'storeId_transfer': pool_1_transfer_clone.iloc[0]['storeId'],
                            'storeName_transfer': pool_1_transfer_clone.iloc[0]['storeName'], 
                            'storeId_receive':    cache_transaction.iloc[0]['storeId'],
                            'storeName_receive':  cache_transaction.iloc[0]['storeName'],
                            'productId':          pool_1_transfer_clone.iloc[0]['productId'],
                            'productName':        pool_1_transfer_clone.iloc[0]['productName'], 'Quantity': n}
                    
                    # Add logs transaction
                    transaction =    pd.concat([transaction,pd.DataFrame(data = e, index=[0])], ignore_index=False)
                    
                    #   Update transfer number and delete row in pool_1
                    pool_2_get_clone.loc[cache_transaction.index[0], ['Balance_num']] = m
                    
                    # Update stockQuantity in  pool_2_get_clone
                    #pool_2_get_clone.loc[cache_transaction.index[0], ['StockQuantity']] = transfering
                    
                    # drop numbers goods transfer in pool_1
                    pool_1_transfer_clone.drop(index=pool_1_transfer_clone.iloc[0:1,:].index ,inplace=True)
                    
                    #   Refresh pool
                    pool_1_transfer_clone = sort_by_balance(pool_1_transfer_clone)
                    pool_2_get_clone = sort_by_balance(pool_2_get_clone)

                    #   Breake when pool_1 is blank
                    if len(pool_1_transfer_clone.index) == 0 :
                        break
                        
                    #   Update cache transaction    
                    cache_transaction=pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
                    continue
                    

            # filter transactions > 0
            transaction = transaction[transaction['Quantity']>0]
            
            # filter goods to HO with quanity > 0
            pool_3_move_to_HO = pool_3_move_to_HO[pool_3_move_to_HO['Quantity']>0]
            
            # define excel out_path - file location and excel writer with engine
            out_path = f"{path}{data.iloc[0]['Area']}.xlsx" #with areas
            writer = pd.ExcelWriter(out_path , engine = 'xlsxwriter')
            
            # export transaction with areas
            transaction.to_excel(writer, sheet_name=f"transfer_{data.iloc[0]['Area']}", encoding='utf-8-sig', index=False)
            
            # export list hàng thiếu hàng
            drop_sub_columns_pool2(pool_2_get_clone).to_excel(writer, sheet_name=f"Statement_{data.iloc[0]['Area']}_thiếuhàng", encoding='utf-8-sig', index=False)
            
            # export list move to HO - Dư hàng
            pool_3_move_to_HO.to_excel(writer, sheet_name = f"Statement_{data.iloc[0]['Area']}_thừahàng", encoding='utf-8-sig', index=False)
            
            pool1_tong_hop = pd.concat([pool1_tong_hop, pool_3_move_to_HO], ignore_index=True)
            
            pool2_tong_hop = pd.concat([pool2_tong_hop, drop_sub_columns_pool2(pool_2_get_clone)], ignore_index=True)
            
            writer.save()
            
        else: # Areas <> HCM, HN
            # Append excess goods
            list_pool_1.append( drop_sub_columns_pool1( pool_1.rename(columns={'Balance_num':'Quantity', 'storeId':'storeId_HO', 'storeName':'storeName_HO'}))) #Thừa hàng
            
            # Append lacked goods
            list_pool_2.append( drop_sub_columns_pool2(pool_2))
            
            # Listing sheets of pool by Areas
            list_sheet_name_pool_1.append(f"{data.iloc[0]['Area']}_HO") #Name for sheet_name in pool_1 # excess goods
            list_sheet_name_pool_2.append(f"HO_{data.iloc[0]['Area']}") #Name for sheet_name in pool_2 # lacked goods

        '''cmt'''

    '''cmt'''
    # define excel writer of excess goods by areas
    writer_pool_1 = pd.ExcelWriter( f"{path}danh sách store tỉnh thừa hàng.xlsx" , engine='xlsxwriter')
    # define excel writer of lacked goods by areas
    writer_pool_2 = pd.ExcelWriter( f"{path}danh sách store tỉnh thiếu hàng.xlsx" , engine='xlsxwriter')
    
    # for loop to create a pool_tong_hop of excess, lacked goods by areas
    for pool1, name_pool1, pool2, name_pool2 in zip(list_pool_1, list_sheet_name_pool_1, list_pool_2, list_sheet_name_pool_2):
        
        pool1.to_excel( writer_pool_1, sheet_name = name_pool1, encoding = 'utf-8-sig', index = False )

        pool2.to_excel( writer_pool_2, sheet_name = name_pool2, encoding = 'utf-8-sig', index = False )
        
        pool1 = pool1.drop(['Area'], axis = 1)
        
        pool1_tong_hop = pd.concat([pool1_tong_hop, pool1], ignore_index = True)
        
        pool2_tong_hop = pd.concat([pool2_tong_hop, pool2], ignore_index = True)
    
    # saving changes
    writer_pool_1.save()
    writer_pool_2.save()
    '''cmt'''
    
    # write pool_tong_hop to excel files
    pool1_tong_hop.to_excel(f'{path}danh sách tất cả store thừa hàng.xlsx', encoding='utf-8-sig', index=False)
    
    pool2_tong_hop.to_excel(f'{path}danh sách tất cả store thiếu hàng.xlsx', encoding='utf-8-sig', index=False)
    
    pool1_tong_hop = pool1_tong_hop.rename(columns={'storeId_HO':'storeId', 'storeName_HO':'storeName', 'Quantity':'StockQuantity'})

    '''cmt'''
    
    ### LOOP 2
    # concat to update stocks in HO inventory from pool_excess_goods
    df_stock_HO = pd.concat([df_1,pool1_tong_hop], ignore_index=True)
    
    # transform HO inventory dataframe
    df_stock_HO = df_stock_HO.groupby(by=['productId', 'productName'])['StockQuantity'].sum()
    df_stock_HO = df_stock_HO.reset_index()
    df_stock_HO['storeId'] = '88003'
    df_stock_HO['storeName'] = 'KHO_DP2'
    df_stock_HO['Area'] = 'KHO_DP2'
    df_stock_HO['SO1'] = 0
    df_stock_HO['SO2'] = 0
    df_stock_HO['SO3'] = 0
    df_stock_HO['SO4'] = 0
    df_stock_HO['AvgSO'] = 0
    df_stock_HO = df_stock_HO.reindex(columns=['Area', 'storeId', 'storeName', 'productId', 'productName', 'SO1', 'SO2', 'SO3', 'SO4', 'AvgSO', 'StockQuantity'])
    
    # writes HO inventory after reviews goods from excess stores
    #df_stock_HO.to_excel(f'{path}KHO_HO_sau_khi_nhận_hàng.xlsx', encoding='utf-8-sig', index=False)
    
    # create dataframe to re-distribute goods to areas - loop 2
    df_stock_HO = pd.concat([df_stock_HO, pool2_tong_hop], ignore_index=True)
    
    # fill np.nan and replace its with 0
    df_stock_HO = df_stock_HO.fillna(np.nan).replace([np.nan], 0)
    
    # clip() to resolve negative numbers
    df_stock_HO['AvgSO'] = df_stock_HO['AvgSO'].clip(lower=0)

    # compute SellPower for new dataframe
    df_stock_HO['SellPower'] = round(df_stock_HO['AvgSO'].div(7),3)
    
    # next, compute DOS 
    df_stock_HO['DOS'] = round(df_stock_HO['StockQuantity'] / df_stock_HO['SellPower'],0)

    # sort values by DOS and identify goods statement, assign Balance_num for goods in HO inventory = 0
    df_stock_HO = df_stock_HO.sort_values( by=['DOS'], ascending = False)
    df_stock_HO['Statement'] = df_stock_HO['DOS'].apply(lambda x: DOS_Classify(x))
    df_stock_HO['Balance_num'] = 0
    
    # for loop to compute realistic Balance_num
    for i,row in df_stock_HO.iterrows():
        val = row['Statement']

        if val == "Thừa hàng":
            df_stock_HO.at[i,'Balance_num']= math.ceil(df_stock_HO['StockQuantity'][i] - df_stock_HO['SellPower'][i] * max_day) 
            
        if val == "Thiếu hàng":
            df_stock_HO.at[i,'Balance_num'] = round(-df_stock_HO['StockQuantity'][i] + df_stock_HO['SellPower'][i] * min_day,0)
            
        if val == "Đủ hàng": df_stock_HO.at[i,'Balance_num'] = 0

    # export df_stock_HO file to xlsx
    df_stock_HO.to_excel(f'{path}stocks_HO_afer.xlsx', encoding = 'utf-8-sig')
    
    # Initialize pool_1 excess goods
    pool_1 = df_stock_HO.loc[df_stock_HO['Statement'] == 'Thừa hàng']
    pool_1 = sort_by_balance(pool_1)

    ## Initialize pool_2 lacked goods
    pool_2 = df_stock_HO.loc[df_stock_HO['Statement'] == 'Thiếu hàng'] 
    pool_2 = sort_by_balance(pool_2)

    # identify new pool_1_transfer_clone ; pool_2_get_clone
    pool_1_transfer_clone = pool_1
    pool_2_get_clone = pool_2

    # Identify
    # First Initialize cache_transaction
    cache_transaction = pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
    
    # create Transaction dataframe
    ## Rule: Compare the productId and Balance number
    transaction = pd.DataFrame(columns=['storeId_transfer','storeName_transfer', 'storeId_receive','storeName_receive', 'productId','productName', 'Quantity'])
    
    # create pool_3_move_to_HO ???
    pool_3_move_to_HO = pd.DataFrame(columns=['storeId_HO', 'storeName_HO', 'productId','productName', 'Quantity'])
    
    while True:
        # The amount of goods exceeded pool_1
        n   =   pool_1_transfer_clone.iloc[0]['Balance_num']
        
        # resolve case when ca_transaction is blank
        if len(cache_transaction.index) == 0  :
            d = {'storeId_HO':  pool_1_transfer_clone.iloc[0]['storeId'],
                'storeName_HO': pool_1_transfer_clone.iloc[0]['storeName'], 
                'productId':          pool_1_transfer_clone.iloc[0]['productId'],
                'productName':        pool_1_transfer_clone.iloc[0]['productName'],
                'Quantity':           n}
            
            # update pool_3_stay_HO
            pool_3_move_to_HO = pd.concat([pool_3_move_to_HO, pd.DataFrame(data = d,index=[0])], ignore_index = False)
            
            # update pool_1 to select another items
            pool_1_transfer_clone.drop(index = pool_1_transfer_clone.iloc[0:1,:].index, inplace = True)
            
            # condition to break loop
            if len(pool_1_transfer_clone.index) == 0:
                break
                
            # upadte cache_transaction
            cache_transaction = pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
            
            continue
            
        # The amount of goods lacked pool_2 
        m   =   cache_transaction.iloc[0]['Balance_num']

        ## Case : if the exceed > lack
        if  n    >=   m :

            transfering = n - m
        # This dataframe is storing the transaction
            c = {'storeId_transfer': pool_1_transfer_clone.iloc[0]['storeId'],
                'storeName_transfer': pool_1_transfer_clone.iloc[0]['storeName'], 
                'storeId_receive': cache_transaction.iloc[0]['storeId'],
                'storeName_receive': cache_transaction.iloc[0]['storeName'],
                'productId': pool_1_transfer_clone.iloc[0]['productId'],
                'productName': pool_1_transfer_clone.iloc[0]['productName'], 'Quantity': m}
            
            # create transaction dataframe
            transaction = pd.concat([transaction,pd.DataFrame(data = c, index=[0])], ignore_index = False)

            #   Migrate the value    
            pool_1_transfer_clone.loc[pool_1_transfer_clone.index[0], ['Balance_num']] = transfering 
            
            pool_2_get_clone.drop(index=cache_transaction.iloc[0:1,:].index, inplace=True)

            #   Refresh
            pool_1_transfer_clone = sort_by_balance(pool_1_transfer_clone)
            pool_2_get_clone = sort_by_balance(pool_2_get_clone)

            #   Update cache_transaction 
            if len(pool_2_get_clone.index)==0:
                break
            cache_transaction=pool_2_get_clone.loc[pool_2_get_clone['productId'] ==
                                                pool_1_transfer_clone.iloc[0]['productId']]
            continue
            
        #Case 2 lack > exceed
        if m > n  :

            transfering = m - n 

            # This dataframe is storing the transaction

            e = {'storeId_transfer': pool_1_transfer_clone.iloc[0]['storeId'],
                    'storeName_transfer': pool_1_transfer_clone.iloc[0]['storeName'], 
                    'storeId_receive': cache_transaction.iloc[0]['storeId'],
                    'storeName_receive': cache_transaction.iloc[0]['storeName'],
                    'productId': pool_1_transfer_clone.iloc[0]['productId'],
                    'productName': pool_1_transfer_clone.iloc[0]['productName'], 'Quantity': n}

            transaction = pd.concat([transaction,pd.DataFrame(data = e,index=[0])], ignore_index=False)
            
            #   Migrate values and delete perious row
            pool_2_get_clone.loc[cache_transaction.index[0], ['Balance_num']] = transfering
        
            pool_1_transfer_clone.drop(index = pool_1_transfer_clone.iloc[0:1,:].index ,inplace=True)
            
            #   Refresh 
            pool_1_transfer_clone = sort_by_balance(pool_1_transfer_clone)
            pool_2_get_clone = sort_by_balance(pool_2_get_clone)

            #   Break loop when pool_1_transfer is blank
            if len(pool_1_transfer_clone.index) == 0 :
                break
            
            # update cache_transaction
            cache_transaction = pool_2_get_clone.loc[pool_2_get_clone['productId'] == pool_1_transfer_clone.iloc[0]['productId']]
            
            continue
    
    # filter transactions > 0
    transaction = transaction[transaction['Quantity'] > 0]
    
    # filter quantity to HO > 0
    pool_3_move_to_HO = pool_3_move_to_HO[pool_3_move_to_HO['Quantity'] > 0]
    
    out_path = f"{path}KHO_HO_FINAL.xlsx"
    
    # create writer
    writer = pd.ExcelWriter(out_path ,engine = 'xlsxwriter')
    
    # save transtactions loop2 as excel sheet
    transaction.to_excel(writer, sheet_name = f"transaction_KHO_HO_FINAL", encoding = 'utf-8-sig', index = False)
    
    # list of lacked items after loop 2
    pool_2_get_clone = pool_2_get_clone[pool_2_get_clone["StockQuantity"] > 0]
    drop_sub_columns_pool2(pool_2_get_clone).to_excel(writer, sheet_name = f"Statement_KHO_FINAL_thiếuhàng", encoding = 'utf-8-sig', index = False)
    
    
    pool_3_move_to_HO.to_excel(writer, sheet_name=f"KHO_FINAL_thừahàng", encoding = 'utf-8-sig', index = False)
    
    pool1_tong_hop=pd.concat([pool1_tong_hop, pool_3_move_to_HO], ignore_index=True)
    
    pool2_tong_hop=pd.concat([pool2_tong_hop, drop_sub_columns_pool2(pool_2_get_clone)], ignore_index=True)
    writer.save()
# ''''cmt'''

    return True
    
    

In [7]:
allotment(df_1, df_2)

True

In [8]:
# raw_df
raw_df = pd.read_excel(f'{path}Stocks.xlsx')
raw_df = raw_df.fillna(0)
raw_df['spId'] = raw_df.apply(lambda row: str(row['storeId']) + str(row['productId']), axis = 1)

In [9]:
# transfer HCM
trans_1_hcm = pd.read_excel(f'{path}HồChíMinh.xlsx', sheet_name = 'transfer_HồChíMinh')
trans_1_hcm['receive_id'] = trans_1_hcm.apply(lambda row: str(row['storeId_receive']) + str(row['productId']), axis = 1)
trans_1_hcm['transfer_id'] = trans_1_hcm.apply(lambda row: str(row['storeId_transfer']) + str(row['productId']), axis = 1)

receive_1_hcm = trans_1_hcm.groupby(by = 'receive_id', axis = 0, as_index = False).sum()

transfer_1_hcm = trans_1_hcm.groupby(by = 'transfer_id', axis = 0, as_index = False).sum()

In [10]:
# transfer HN
trans_1_hn = pd.read_excel(f'{path}HàNội.xlsx', sheet_name = 'transfer_HàNội')
trans_1_hn['receive_id'] = trans_1_hn.apply(lambda row: str(row['storeId_receive']) + str(row['productId']), axis = 1)
trans_1_hn['transfer_id'] = trans_1_hn.apply(lambda row: str(row['storeId_transfer']) + str(row['productId']), axis = 1)
receive_1_hn = trans_1_hn.groupby(by = 'receive_id', axis = 0, as_index = False).sum()
transfe_1_hn = trans_1_hn.groupby(by = 'transfer_id', axis = 0, as_index = False).sum()

In [11]:
# move to HO - loop 1
move_1_HO = pd.read_excel(f'{path}danh sách tất cả store thừa hàng.xlsx')
move_1_HO['move_1_HO_id'] = move_1_HO.apply(lambda row: str(row['storeId_HO']) + str(row['productId']), axis = 1)

In [12]:
# receive from HO - loop 2
HO_to_store = pd.read_excel(f'{path}KHO_HO_FINAL.xlsx', sheet_name = 'transaction_KHO_HO_FINAL')
HO_to_store['trans_2_id'] = HO_to_store.apply(lambda row: str(row['storeId_receive']) + str(row['productId']), axis = 1)

In [13]:
raw_df = raw_df.merge(receive_1_hcm.loc[:,['receive_id','Quantity']], how = 'left', left_on = 'spId', right_on = 'receive_id')
raw_df = raw_df.drop('receive_id', axis = 1)
raw_df = raw_df.rename(columns = {'Quantity': 'hcm_1_rec'})

In [14]:
raw_df = raw_df.merge(transfer_1_hcm.loc[:,['transfer_id','Quantity']], how = 'left', left_on = 'spId', right_on = 'transfer_id')
raw_df = raw_df.drop('transfer_id', axis = 1)
raw_df = raw_df.rename(columns = {'Quantity': 'hcm_1_trans'})

In [15]:
raw_df = raw_df.merge(receive_1_hn.loc[:,['receive_id','Quantity']], how = 'left', left_on = 'spId', right_on = 'receive_id')
raw_df = raw_df.drop('receive_id', axis = 1)
raw_df = raw_df.rename(columns = {'Quantity': 'hn_1_rec'})

In [16]:
raw_df = raw_df.merge(transfe_1_hn.loc[:,['transfer_id','Quantity']], how = 'left', left_on = 'spId', right_on = 'transfer_id')
raw_df = raw_df.drop('transfer_id', axis = 1)
raw_df = raw_df.rename(columns = {'Quantity': 'hn_1_trans'})

In [17]:
raw_df = raw_df.merge(move_1_HO.loc[:,['move_1_HO_id','Quantity']], how = 'left', left_on = 'spId', right_on = 'move_1_HO_id')
raw_df = raw_df.drop('move_1_HO_id', axis = 1)
raw_df = raw_df.rename(columns = {'Quantity': 'move_1_HO'})

In [18]:
raw_df = raw_df.merge(HO_to_store.loc[:,['trans_2_id','Quantity']], how = 'left', left_on = 'spId', right_on = 'trans_2_id')
raw_df = raw_df.drop('trans_2_id', axis = 1)
raw_df = raw_df.rename(columns = {'Quantity': 'trans_2'})

In [19]:
raw_df = raw_df.fillna(0)
raw_df['final_quantity'] = raw_df['StockQuantity'] + raw_df['hcm_1_rec'] - raw_df['hcm_1_trans'] + raw_df['hn_1_rec'] \
                            - raw_df['hn_1_trans'] - raw_df['move_1_HO'] + raw_df['trans_2']
raw_df

,Area,storeId,storeName,brandName,Cat2022,productId,productName,StockQuantity,SO1,SO2,...,SO4,AvgSO,spId,hcm_1_rec,hcm_1_trans,hn_1_rec,hn_1_trans,move_1_HO,trans_2,final_quantity
0,Đà Nẵng,101856,DDV ĐN 60 Hàm Nghi,Apple,IPAD MỚI,50000198,"APPLE IPAD 2021 10.2"" WIFI 64GB (CTY) - Bạc MK...",2.0,0.0,1.0,...,2.0,1.00,10185650000198,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,Đà Nẵng,101856,DDV ĐN 60 Hàm Nghi,Apple,IPAD MỚI,50000199,"APPLE IPAD 2021 10.2"" WIFI 64GB (CTY) - Xám MK...",0.0,3.0,2.0,...,3.0,2.50,10185650000199,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Đà Nẵng,101856,DDV ĐN 60 Hàm Nghi,Apple,IPAD MỚI,50011648,"APPLE IPAD 2022 10.9"" WIFI 64GB (CTY) - Xanh M...",0.0,0.0,0.0,...,0.0,0.50,10185650011648,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Đà Nẵng,101856,DDV ĐN 60 Hàm Nghi,Apple,IPAD MỚI,50011649,"APPLE IPAD 2022 10.9"" WIFI 64GB (CTY) - Hồng M...",0.0,0.0,0.0,...,1.0,0.25,10185650011649,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Đà Nẵng,101856,DDV ĐN 60 Hàm Nghi,Apple,IPAD MỚI,50011650,"APPLE IPAD 2022 10.9"" WIFI 64GB (CTY) - Vàng M...",0.0,0.0,0.0,...,0.0,0.25,10185650011650,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274,Miền Tây,101862,DDV MT 160 30TH4,XiaoMi,"XIAOMI, NOKIA",50014617,XIAOMI REDMI 12C 4GB/64GB(CTY) - Xanh lá cây -...,1.0,0.0,1.0,...,0.0,0.25,10186250014617,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5275,Miền Tây,101862,DDV MT 160 30TH4,XiaoMi,"XIAOMI, NOKIA",50014618,XIAOMI REDMI 12C 4GB/64GB(CTY) - Xám - New,2.0,0.0,0.0,...,0.0,0.25,10186250014618,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5276,Miền Tây,101862,DDV MT 160 30TH4,XiaoMi,"XIAOMI, NOKIA",50015167,XIAOMI REDMI NOTE 12 4GB/128GB(CTY) - Xanh dươ...,4.0,0.0,1.0,...,0.0,0.25,10186250015167,0.0,0.0,0.0,0.0,3.0,0.0,1.0
5277,Miền Tây,101862,DDV MT 160 30TH4,XiaoMi,"XIAOMI, NOKIA",50015168,XIAOMI REDMI NOTE 12 4GB/128GB(CTY) - Xám - New,7.0,0.0,0.0,...,0.0,0.00,10186250015168,0.0,0.0,0.0,0.0,7.0,0.0,0.0


In [20]:
raw_df.to_excel(f'{path}finalStocks.xlsx', encoding = 'utf-8-sig')